In [25]:
import numpy as np
import pandas as pd

from pathlib import Path

# Graph Cleanup
This notebook puts data in a convenient format for creating the final graph.

In [26]:
DATAPATH = Path('../data/processed/')
PRPATH = DATAPATH / 'pagerank_rev_edge.parquet' #Can change this depending on which pagerank output you want
KYCPATH = DATAPATH / 'kyc.parquet'
EPATH = DATAPATH / 'emt.parquet'
WPATH = DATAPATH / 'wire.parquet'

In [29]:
# Load all nodes
node_df = pd.read_parquet(PRPATH)

# Join with kyc data and add country column
kyc_df = pd.read_parquet(KYCPATH)
kyc_df['country'] = 'CA'
node_df = pd.merge(node_df, kyc_df, how='left', on='cust_id')

# Get additional countries from wiretransfer data
wdf = pd.read_parquet(WPATH)
s1 = wdf[['cust_id_sender', 'country_sender']].copy().rename(columns={'cust_id_sender':'cust_id', 'country_sender':'country'})
s2 = wdf[['cust_id_receiver', 'country_receiver']].copy().rename(columns={'cust_id_receiver':'cust_id', 'country_receiver':'country'})
countries = pd.concat([s1,s2])
countries = countries.drop_duplicates()

node_df = pd.merge(node_df, countries, how='left', on='cust_id')
node_df['country_x'] = node_df['country_x'].combine_first(node_df['country_y'])
node_df['country'] = node_df['country_x']
node_df = node_df.drop(columns=['country_x', 'country_y'])

#exporting 
node_df.to_parquet(DATAPATH / 'nodes.parquet', index=False)
print(f"Node list length: {len(node_df)}")

Node list length: 300008
